In [2]:
import pandas as pd
import h5py

In [ ]:
FILE_PATH = "/Users/sachetz/uchicago_mpcs/big_data_app_arch/A/A/A/TRAAAAK128F9318786.h5"

# File Structure

In [4]:
def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"Group: {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name} | Shape: {obj.shape} | Data type: {obj.dtype}")

with h5py.File(FILE_PATH, 'r') as h5_file:
    h5_file.visititems(print_structure)

Group: analysis
Dataset: analysis/bars_confidence | Shape: (87,) | Data type: float64
Dataset: analysis/bars_start | Shape: (87,) | Data type: float64
Dataset: analysis/beats_confidence | Shape: (350,) | Data type: float64
Dataset: analysis/beats_start | Shape: (350,) | Data type: float64
Dataset: analysis/sections_confidence | Shape: (6,) | Data type: float64
Dataset: analysis/sections_start | Shape: (6,) | Data type: float64
Dataset: analysis/segments_confidence | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_max | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_max_time | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_start | Shape: (585,) | Data type: float64
Dataset: analysis/segments_pitches | Shape: (585, 12) | Data type: float64
Dataset: analysis/segments_start | Shape: (585,) | Data type: float64
Dataset: analysis/segments_timbre | Shape: (585, 12) | Data type: float64
Dataset: analysis/songs | Shape: (1,)

# Extracting to Pandas Dataframe

In [5]:
h5 = h5py.File(FILE_PATH, 'r')
store = pd.HDFStore(FILE_PATH, mode = 'r')

In [6]:
songs_metadata = store.get("metadata/songs")
songs_metadata

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,166043,,0,0,Adelitas Way,497103,0.733372,SOBLFFE12AF72AA5BA,Scream,5504670


In [7]:
songs_analysis = store.get("analysis/songs")
songs_analysis

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,7b6557889cc46b202300a4baf902e229,0.0,213.9424,0.0,0.0,0,0,0,0,...,1,0.349,-4.769,1,0.403,210.048,99.944,4,0.846,TRAAAAK128F9318786


In [8]:
songs_year = store.get("musicbrainz/songs")
songs_year

,idx_artist_mbtags,year
0,0,2009


In [9]:
bars_confidence = pd.Series([[item for item in list(h5.get("analysis/bars_confidence"))]], name = "bars_confidence")
bars_confidence

0    [0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...
Name: bars_confidence, dtype: object

In [10]:
segments_pitches = pd.Series([[item for item in list(h5.get("analysis/segments_pitches"))]], name = "segments_pitches")
segments_pitches

0    [[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18...
Name: segments_pitches, dtype: object

In [11]:
song = pd.concat([songs_metadata, songs_analysis, songs_year, bars_confidence, segments_pitches], axis=1, join='outer')
song

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year,bars_confidence,segments_pitches
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,...,0.403,210.048,99.944,4,0.846,TRAAAAK128F9318786,0,2009,"[0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...","[[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18..."


In [12]:
song = pd.concat([
    store.get("metadata/songs"),
    store.get("analysis/songs"),
    store.get("musicbrainz/songs"),
    pd.Series([[item for item in list(h5.get("analysis/bars_confidence"))]], name = "bars_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/bars_start"))]], name = "bars_start"),
    pd.Series([[item for item in list(h5.get("analysis/beats_confidence"))]], name = "beats_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/beats_start"))]], name = "beats_start"),
    pd.Series([[item for item in list(h5.get("analysis/sections_confidence"))]], name = "sections_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/sections_start"))]], name = "sections_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_confidence"))]], name = "segments_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_max"))]], name = "segments_loudness_max"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_max_time"))]], name = "segments_loudness_max_time"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_start"))]], name = "segments_loudness_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_pitches"))]], name = "segments_pitches"),
    pd.Series([[item for item in list(h5.get("analysis/segments_start"))]], name = "segments_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_timbre"))]], name = "segments_timbre"),
    pd.Series([[item for item in list(h5.get("analysis/tatums_confidence"))]], name = "tatums_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/tatums_start"))]], name = "tatums_start"),
    pd.Series([[item.decode() for item in list(h5.get("metadata/artist_terms"))]], name = "artist_terms"),
    pd.Series([[item for item in list(h5.get("metadata/artist_terms_freq"))]], name = "artist_terms_freq"),
    pd.Series([[item for item in list(h5.get("metadata/artist_terms_weight"))]], name = "artist_terms_weight"),
    pd.Series([[item.decode() for item in list(h5.get("metadata/similar_artists"))]], name = "similar_artists"),
    pd.Series([[item for item in list(h5.get("musicbrainz/artist_mbtags"))]], name = "artist_mbtags"),
    pd.Series([[item for item in list(h5.get("musicbrainz/artist_mbtags_count"))]], name = "artist_mbtags_count")
], axis=1, join='outer')
song.columns

Index(['analyzer_version', 'artist_7digitalid', 'artist_familiarity',
       'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location',
       'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid',
       'genre', 'idx_artist_terms', 'idx_similar_artists', 'release',
       'release_7digitalid', 'song_hotttnesss', 'song_id', 'title',
       'track_7digitalid', 'analysis_sample_rate', 'audio_md5', 'danceability',
       'duration', 'end_of_fade_in', 'energy', 'idx_bars_confidence',
       'idx_bars_start', 'idx_beats_confidence', 'idx_beats_start',
       'idx_sections_confidence', 'idx_sections_start',
       'idx_segments_confidence', 'idx_segments_loudness_max',
       'idx_segments_loudness_max_time', 'idx_segments_loudness_start',
       'idx_segments_pitches', 'idx_segments_start', 'idx_segments_timbre',
       'idx_tatums_confidence', 'idx_tatums_start', 'key', 'key_confidence',
       'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo',

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(song)

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,idx_sections_confidence,idx_sections_start,idx_segments_confidence,idx_segments_loudness_max,idx_segments_loudness_max_time,idx_segments_loudness_start,idx_segments_pitches,idx_segments_start,idx_segments_timbre,idx_tatums_confidence,idx_tatums_start,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year,bars_confidence,bars_start,beats_confidence,beats_start,sections_confidence,sections_start,segments_confidence,segments_loudness_max,segments_loudness_max_time,segments_loudness_start,segments_pitches,segments_start,segments_timbre,tatums_confidence,tatums_start,artist_terms,artist_terms_freq,artist_terms_weight,similar_artists,artist_mbtags,artist_mbtags_count
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,166043,,0,0,Adelitas Way,497103,0.733372,SOBLFFE12AF72AA5BA,Scream,5504670,22050,7b6557889cc46b202300a4baf902e229,0.0,213.9424,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.349,-4.769,1,0.403,210.048,99.944,4,0.846,TRAAAAK128F9318786,0,2009,"[0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...","[0.10184, 2.61136, 5.11211, 7.60756, 10.10465,...","[0.0, 0.359, 0.282, 0.154, 0.023, 0.168, 0.056...","[0.10184, 0.73158, 1.35706, 1.99126, 2.61136, ...","[1.0, 0.946, 0.417, 0.416, 0.305, 0.268]","[0.0, 8.2306, 19.37426, 42.15187, 68.56066, 20...","[1.0, 0.077, 0.037, 0.168, 0.222, 0.489, 0.001...","[-20.556, -21.139, -20.528, -20.028, -20.774, ...","[0.11934, 0.02379, 0.097, 0.02208, 0.02019, 0....","[-60.0, -22.718, -23.017, -22.855, -22.356, -2...","[[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18...","[0.0, 0.54286, 0.70553, 1.48367, 1.68063, 1.95...","[[17.977, 71.335, 193.384, -323.599, 25.153, 7...","[0.429, 0.39, 0.355, 0.328, 0.303, 0.277, 0.24...","[0.10184, 0.25966, 0.41592, 0.57454, 0.73158, ...","[alternative metal, heavy metal, rock, alterna...","[0.9958746574225641, 0.9958746574225641, 1.0, ...","[1.0, 0.8993537393186563, 0.7561876828863154, ...","[AROQMHX11C8A42BD89, ARZEKVZ1187FB51D85, ARVFR...",[],[]


In [17]:
columns_to_use = [
    "analysis_sample_rate",
    "artist_7digitalid",
    "artist_familiarity",
    "artist_hotttnesss",
    "artist_id",
    "artist_latitude",
    "artist_location",
    "artist_longitude",
    "artist_mbid",
    "artist_mbtags",
    "artist_mbtags_count",
    "artist_name",
    "artist_playmeid",
    "artist_terms",
    "artist_terms_freq",
    "artist_terms_weight",
    "audio_md5",
    "bars_confidence",
    "bars_start",
    "beats_confidence",
    "beats_start",
    "danceability",
    "duration",
    "end_of_fade_in",
    "energy",
    "key",
    "key_confidence",
    "loudness",
    "mode",
    "mode_confidence",
    "release",
    "release_7digitalid",
    "sections_confidence",
    "sections_start",
    "segments_confidence",
    "segments_loudness_max",
    "segments_loudness_max_time",
    "segments_loudness_start",
    "segments_pitches",
    "segments_start",
    "segments_timbre",
    "similar_artists",
    "song_hotttnesss",
    "song_id",
    "start_of_fade_out",
    "tatums_confidence",
    "tatums_start",
    "tempo",
    "time_signature",
    "time_signature_confidence",
    "title",
    "track_id",
    "track_7digitalid",
    "year"
]
song = song[columns_to_use]
song

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_mbtags,...,start_of_fade_out,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,title,track_id,track_7digitalid,year
0,22050,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,[],...,210.048,"[0.429, 0.39, 0.355, 0.328, 0.303, 0.277, 0.24...","[0.10184, 0.25966, 0.41592, 0.57454, 0.73158, ...",99.944,4,0.846,Scream,TRAAAAK128F9318786,5504670,2009


In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(song)

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_mbtags,artist_mbtags_count,artist_name,artist_playmeid,artist_terms,artist_terms_freq,artist_terms_weight,audio_md5,bars_confidence,bars_start,beats_confidence,beats_start,danceability,duration,end_of_fade_in,energy,key,key_confidence,loudness,mode,mode_confidence,release,release_7digitalid,sections_confidence,sections_start,segments_confidence,segments_loudness_max,segments_loudness_max_time,segments_loudness_start,segments_pitches,segments_start,segments_timbre,similar_artists,song_hotttnesss,song_id,start_of_fade_out,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,title,track_id,track_7digitalid,year
0,22050,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,[],[],Adelitas Way,166043,"[alternative metal, heavy metal, rock, alterna...","[0.9958746574225641, 0.9958746574225641, 1.0, ...","[1.0, 0.8993537393186563, 0.7561876828863154, ...",7b6557889cc46b202300a4baf902e229,"[0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...","[0.10184, 2.61136, 5.11211, 7.60756, 10.10465,...","[0.0, 0.359, 0.282, 0.154, 0.023, 0.168, 0.056...","[0.10184, 0.73158, 1.35706, 1.99126, 2.61136, ...",0.0,213.9424,0.0,0.0,1,0.349,-4.769,1,0.403,Adelitas Way,497103,"[1.0, 0.946, 0.417, 0.416, 0.305, 0.268]","[0.0, 8.2306, 19.37426, 42.15187, 68.56066, 20...","[1.0, 0.077, 0.037, 0.168, 0.222, 0.489, 0.001...","[-20.556, -21.139, -20.528, -20.028, -20.774, ...","[0.11934, 0.02379, 0.097, 0.02208, 0.02019, 0....","[-60.0, -22.718, -23.017, -22.855, -22.356, -2...","[[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18...","[0.0, 0.54286, 0.70553, 1.48367, 1.68063, 1.95...","[[17.977, 71.335, 193.384, -323.599, 25.153, 7...","[AROQMHX11C8A42BD89, ARZEKVZ1187FB51D85, ARVFR...",0.733372,SOBLFFE12AF72AA5BA,210.048,"[0.429, 0.39, 0.355, 0.328, 0.303, 0.277, 0.24...","[0.10184, 0.25966, 0.41592, 0.57454, 0.73158, ...",99.944,4,0.846,Scream,TRAAAAK128F9318786,5504670,2009


In [1]:
import pandas as pd
df = pd.read_parquet("/Users/sachetz/uchicago_mpcs/big_data_app_arch/million-song-recommendation/A.parquet")
df.shape[0]

53